In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras import layers, models, optimizers
from keras.datasets import mnist
from keras.utils import to_categorical

# Załaduj dane MNIST
(train_images, train_labels), (_, _) = mnist.load_data()

# Normalizacja danych
train_images = (train_images.astype(np.float32) - 127.5) / 127.5
train_images = np.expand_dims(train_images, axis=-1)

# Konwertuj etykiety na one-hot encoding
train_labels = to_categorical(train_labels, num_classes=10)

# Parametry modelu cGAN
latent_dim = 100

# Funkcja generująca losowy szum dla generatora
def generate_latent_points(latent_dim, n_samples, num_classes):
    latent_points = np.random.randn(n_samples, latent_dim)
    labels = np.random.randint(0, num_classes, n_samples)
    return latent_points, labels

# Funkcja budująca generator
def build_generator(latent_dim, num_classes):
    model = models.Sequential()

    model.add(layers.Dense(128, input_dim=latent_dim + num_classes, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(784, activation='tanh'))
    model.add(layers.Reshape((28, 28, 1)))

    return model

# Funkcja budująca dyskryminator
def build_discriminator(img_shape, num_classes):
    model = models.Sequential()

    model.add(layers.Flatten(input_shape=img_shape))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

# Funkcja budująca model cGAN
def build_cgan(generator, discriminator):
    discriminator.trainable = False

    model = models.Sequential()
    model.add(generator)
    model.add(discriminator)

    return model

# Funkcja trenująca cGAN
def train_cgan(generator, discriminator, cgan, epochs, batch_size, latent_dim, num_classes):
    for epoch in range(epochs):
        # Wygeneruj losowe dane do treningu
        latent_points, labels = generate_latent_points(latent_dim, batch_size, num_classes)

        # Wygeneruj fałszywe obrazy
        generated_images = generator.predict(np.hstack((latent_points, to_categorical(labels, num_classes))))

        # Oznacz obrazy jako prawdziwe (1) lub fałszywe (0)
        real_labels = np.ones((batch_size, 1))
        fake_labels = np.zeros((batch_size, 1))

        # Trenuj dyskryminator
        d_loss_real = discriminator.train_on_batch(np.hstack((train_images[:batch_size], train_labels[:batch_size])), real_labels)
        d_loss_fake = discriminator.train_on_batch(np.hstack((generated_images, to_categorical(labels, num_classes))), fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Trenuj generator przez cGAN
        g_loss = cgan.train_on_batch(np.hstack((latent_points, to_categorical(labels, num_classes))), real_labels)

        # Wyświetl postęp co 100 epok
        if epoch % 100 == 0:
            print(f"Epoka {epoch}, Strata dyskryminatora: {d_loss[0]}, Strata generatora: {g_loss}")

# Tworzenie modeli
generator = build_generator(latent_dim, 10)
discriminator = build_discriminator((28, 28, 1), 10)
cgan = build_cgan(generator, discriminator)

# Kompilacja modeli
discriminator.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(), metrics=['accuracy'])
cgan.compile(loss='binary_crossentropy', optimizer=optimizers.Adam())

# Trenowanie cGAN
train_cgan(generator, discriminator, cgan, epochs=5000, batch_size=64, latent_dim=latent_dim, num_classes=10)


2023-11-24 20:32:49.403452: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-24 20:32:49.754410: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-24 20:32:49.754444: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-24 20:32:49.755761: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-24 20:32:49.936530: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-24 20:32:49.942852: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

2/2 [==============================] - 0s 14ms/step


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 4 dimension(s) and the array at index 1 has 2 dimension(s)